In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import os

import requests
from bs4 import BeautifulSoup

import time
import re
import nltk
from nltk.tokenize import sent_tokenize,word_tokenize

In [2]:
os.chdir(r"D:\YoShope\drive-download-20221112T083709Z-001")
os.listdir()

['Article.txt',
 'dicts',
 'heading.txt',
 'Input.xlsx',
 'LoughranMcDonald_MasterDictionary_2020.csv',
 'Objective.docx',
 'Output Data Structure.xlsx',
 'stopwords.txt',
 'StopWords_Generic.txt',
 'Text Analysis.docx',
 '~$Output Data Structure.xlsx',
 '~$xt Analysis.docx']

In [3]:
df = pd.read_excel("Input.xlsx")
df.head()

,URL_ID,URL
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...
1,38.0,https://insights.blackcoffer.com/what-if-the-c...
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...
3,40.0,https://insights.blackcoffer.com/will-machine-...
4,41.0,https://insights.blackcoffer.com/will-ai-repla...


## On 1 link first

In [4]:
link1 = df["URL"][0]
headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
page = requests.get(link1 , headers=headers).text

soup = BeautifulSoup(page,'html.parser')


In [5]:
# function for flatening of List into a Single list
def flat(lis):
    flatList = []
    # Iterate with outer list
    for element in lis:
        if type(element) is list:
            # Check if type is list than iterate through the sublist
            for item in element:
                flatList.append(item)
        else:
            flatList.append(element)
    return flatList

In [6]:
## stop words
## Generic stop world from github https://gist.github.com/larsyencken/1440509
## https://web.archive.org/web/20220120132311/https://sraf.nd.edu/textual-analysis/resources/#StopWords

def stopwords_list():
    
    stop_words = set()
    
    with open('stopwords.txt','r') as f:
        words = f.readlines()
        for i in words:
            stop_words.add(i.rstrip("\n"))
    
    with open('StopWords_Generic.txt','r') as f:
        lines = f.readlines()
        for i in lines:
            stop_words.add(i.lower().strip())
    
    from nltk.corpus import stopwords
    for i in stopwords.words('english'):
        stop_words.add(i)
    
    return list(stop_words)

In [7]:
stopwords_list()

['full',
 'shall',
 'say',
 'what',
 'doing',
 'those',
 'shouldn',
 'many',
 'may',
 'keeps',
 'one',
 'w',
 'we',
 'herself',
 'is',
 'either',
 'y',
 'between',
 'saw',
 'quite',
 'fully',
 'asks',
 'o',
 'asking',
 'knew',
 'like',
 'enough',
 'your',
 'used',
 'member',
 'even',
 'me',
 'needn',
 'c',
 'ways',
 'to',
 'wells',
 'do',
 'side',
 'presenting',
 'different',
 'today',
 'next',
 'states',
 'backs',
 'give',
 'generally',
 'fact',
 'latest',
 'cases',
 'certain',
 'downs',
 "you've",
 'problem',
 'ain',
 'not',
 'been',
 'b',
 'per',
 'greatest',
 "mightn't",
 'only',
 'her',
 'least',
 'gives',
 'until',
 'parts',
 'some',
 "it's",
 'anywhere',
 'largely',
 "hadn't",
 'necessary',
 'likely',
 'however',
 'thus',
 'back',
 'open',
 'thing',
 'asked',
 'worked',
 'my',
 'members',
 'almost',
 'ending',
 'more',
 'and',
 'his',
 'clear',
 'highest',
 'three',
 'numbers',
 'too',
 'f',
 'this',
 'certainly',
 'sure',
 'best',
 'came',
 'now',
 'with',
 'right',
 "hasn't",


In [8]:
#using LoughranMcDonald_MasterDictionary_2020 of words
#dict link https://sraf.nd.edu/textual-analysis/resources/ 

if not os.path.isdir('dicts'):
    os.mkdir('dicts')
    
def dictinoaries():
    
    #importing stop Words
    stop_words=stopwords_list()
    
    #importing Dictionary
    pos_neg_dict=pd.read_csv('LoughranMcDonald_MasterDictionary_2020.csv')
    
    #Words to lower
    pos_neg_dict['Word']=pos_neg_dict['Word'].apply(lambda row:str(row).lower()) #words to lower
    
    cstm_pos_dict={}
    cstm_neg_dict={}
    
    #making negative word dictionary
    with open('dicts//neg_dict.txt','w') as f:
        for word,score in zip(pos_neg_dict['Word'],pos_neg_dict['Negative']):
            if word not in stop_words and score!=0:
                cstm_neg_dict[word]=-1
                f.write(word+'\n')
            
    #making postive words dictionary
    with open('dicts//pos_dict.txt','w') as f:
        for word,score in zip(pos_neg_dict['Word'],pos_neg_dict['Positive']):
            if word not in stop_words and score!=0:
                cstm_pos_dict[word]=+1
                f.write(word+'\n')
                
    return cstm_neg_dict,cstm_pos_dict

In [9]:
#function to extract positive words from the text which are in the dict and not in the stop words list
#and calculating positive and negative score

def extraction(text):
    
    # importing dictinoary
    cstm_neg_dict, cstm_pos_dict = dictinoaries()
    
    #importing stopwords
    stop_words=stopwords_list()
    
#   cleaning the text
    text = re.sub(r"[^a-zA-Z0-9 ]", "",text.lower() )
    
    ##tokanization
    #nltk.download('punkt')
    
    #declaring tokanizer for Words
    tokenizer =nltk.RegexpTokenizer('\w+')
    #from nltk.tokenize import word_tokenize
    
    #making tokens
    token_text=tokenizer.tokenize(text)
    
    # positive score
    pos_score_dict={}
    for word in token_text:
         if word not in stop_words and word in cstm_pos_dict:
                if word not in pos_score_dict:
                    pos_score_dict[word]=1
                else:
                    pos_score_dict[word]+=1
                
    
    #negtive_score
    neg_score_dict={}
    for word in token_text:
         if word not in stop_words and word in cstm_neg_dict:
                if word not in neg_score_dict:
                    neg_score_dict[word] = -1  # if word not present make 1, 
                else:
                    neg_score_dict[word]-=1  # id present add 1 value to it

    
    #adding all the scores
    text_pos_score=sum(pos_score_dict.values())
    text_neg_score=sum(neg_score_dict.values())*-1
    
    #POLARITY SCORE
    text_polarity_score=(text_pos_score-text_neg_score)/((text_pos_score + text_neg_score) + 0.000001)
    
    #Subjectivity Score: 
    clean_text=[word for word in token_text if word not in stop_words]
    #print('clean_text ',len(clean_text))
    #a=' '.join(str(word) for word in clean_text) in str spaces are also counted in len
    
    subjectivity_score=(text_pos_score + text_neg_score)/(len(clean_text)+ 0.000001)
    #print('subjectivity_score', subjectivity_score)
    
    return text_pos_score,text_neg_score,text_polarity_score,subjectivity_score


In [10]:
def scrap(link):
    
    
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
    page = requests.get(link , headers=headers).text
    
    soup = BeautifulSoup(page,'html.parser')
    
    #extrcting headings
    h1 = soup.find('h1').text
    
    #savinf it in file
    with open("heading.txt", "a") as myfile:
        myfile.write(h1)
        myfile.write("\n")
    
    paragraph =[]
    
    # extarcting article
    for i in soup.find_all("p"):
        paragraph.append(i.text.lower())
    
    straight_para = flat(paragraph)
        
    with open("Article.txt",'a') as f:
        f.write(str(straight_para))
        f.write("\n")
        
    ## 
    sen_count = 0
    word_count=0
    
    sents = []
    words = []
    
    for i in straight_para: 
        sents.append(sent_tokenize(i))
        words.append(word_tokenize(i))
    
        #flattening
        flatList_sent = flat(sents)
        flatList_word = flat(words)

        #Average Number of Words Per Sentence = the total number of words / the total number of sentences
        avg_words_per_sen_length = round(len(flatList_word)/len(flatList_sent),2)
        
        word_count = 0
        for i in flatList_word:
            if i not in stopwords_list():
                word_count+=1
        
    text_pos_score,text_neg_score,text_polarity_score,subjectivity_score = extraction(str(straight_para))
        
        
    return(avg_words_per_sen_length ,word_count ,text_pos_score,text_neg_score,text_polarity_score,subjectivity_score)
    
    
    

    print(len(flatList_word))
    
    
scrap(r"https://insights.blackcoffer.com/how-machine-learning-will-affect-your-business/")

(23.21, 379, 10, 6, 0.24999998437500096, 0.051948051779389444)

## final

In [11]:
data = pd.DataFrame()
data["URL_ID"] = df["URL_ID"]
data["URL"] = df["URL"]

avg_words_per_sentence = []
word_count=[]
text_pos_score = []
text_neg_score =[]
text_polarity=[]
subjectivity_score =[]
success = []

for i in tqdm(df["URL"]):
    try:
        avg_words_per_sent , word_cnt, pos_score,neg_score,text_pol,subj_score = scrap(i)
        
        avg_words_per_sentence.append(avg_words_per_sent)
        word_count.append(word_cnt)
        text_pos_score.append(pos_score)
        text_neg_score.append(neg_score)
        text_polarity.append(text_pol)
        subjectivity_score.append(subj_score)
        success.append("Success")
        
    except:
        success.append("Failed")
        avg_words_per_sentence.append(np.NAN)
        word_count.append(np.NAN)
        text_pos_score.append(np.NAN)
        text_neg_score.append(np.NAN)
        text_polarity.append(np.NAN)
        subjectivity_score.append(np.NAN)
        
    d = {"Response":success,
         "Postive score":text_pos_score,
         "Negatice Score":text_neg_score,
         "Text Polarity":text_polarity,
         "Subjective Score":subjectivity_score,
         "Word Count":word_count,
         "Avg Words Per Sentence":avg_words_per_sentence}
    
    dummy_df = pd.DataFrame(d)
    
data = data.merge(dummy_df , left_index=True , right_index=True)

display(data)

100%|████████████████████████████████████████████████████████████████████████████████| 114/114 [17:56<00:00,  9.44s/it]


,URL_ID,URL,Response,Postive score,Negatice Score,Text Polarity,Subjective Score,Word Count,Avg Words Per Sentence
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...,Success,21.0,17.0,0.105263,0.037438,1225.0,24.58
1,38.0,https://insights.blackcoffer.com/what-if-the-c...,Success,21.0,32.0,-0.207547,0.089679,791.0,20.87
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...,Success,26.0,32.0,-0.103448,0.068720,1045.0,21.10
3,40.0,https://insights.blackcoffer.com/will-machine-...,Success,23.0,19.0,0.095238,0.060172,873.0,17.30
4,41.0,https://insights.blackcoffer.com/will-ai-repla...,Success,25.0,23.0,0.041667,0.059259,1010.0,23.48
...,...,...,...,...,...,...,...,...,...
109,146.0,https://insights.blackcoffer.com/blockchain-fo...,Success,9.0,17.0,-0.307692,0.058296,540.0,19.22
110,147.0,https://insights.blackcoffer.com/the-future-of...,Success,20.0,8.0,0.428571,0.050817,674.0,25.42
111,148.0,https://insights.blackcoffer.com/big-data-anal...,Success,10.0,39.0,-0.591837,0.085965,696.0,19.02
112,149.0,https://insights.blackcoffer.com/business-anal...,Success,13.0,4.0,0.529412,0.060932,335.0,23.69


In [14]:
data.to_excel("output_desired.xlsx")

new_data = pd.read_excel("output_desired.xlsx")
new_data.head()

,Unnamed: 0,URL_ID,URL,Response,Postive score,Negatice Score,Text Polarity,Subjective Score,Word Count,Avg Words Per Sentence
0,0,37,https://insights.blackcoffer.com/ai-in-healthc...,Success,21.0,17.0,0.105263,0.037438,1225.0,24.58
1,1,38,https://insights.blackcoffer.com/what-if-the-c...,Success,21.0,32.0,-0.207547,0.089679,791.0,20.87
2,2,39,https://insights.blackcoffer.com/what-jobs-wil...,Success,26.0,32.0,-0.103448,0.068720,1045.0,21.10
3,3,40,https://insights.blackcoffer.com/will-machine-...,Success,23.0,19.0,0.095238,0.060172,873.0,17.30
4,4,41,https://insights.blackcoffer.com/will-ai-repla...,Success,25.0,23.0,0.041667,0.059259,1010.0,23.48
